<a href="https://colab.research.google.com/github/ranjita782/python-projects/blob/main/loantap2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1ZPYj7CZCfxntE8p2Lze_4QO4MyEOy6_d

Downloading...
From: https://drive.google.com/uc?id=1ZPYj7CZCfxntE8p2Lze_4QO4MyEOy6_d
To: /content/logistic_regression.csv
100% 100M/100M [00:04<00:00, 21.3MB/s] 


In [ ]:
#Header files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import gdown as gd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import math
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

In [ ]:
# Import the dataset
data = pd.read_csv('logistic_regression.csv')

In [ ]:
# Example of imputation for emp_title
data['emp_title'].fillna('Unknown', inplace=True)

data['emp_length'].fillna('Unknown', inplace=True)


In [ ]:
data['revol_util'].fillna(data['revol_util'].median(), inplace=True)


In [ ]:
# Example of correlation-based imputation
total_acc_avg = data.groupby('total_acc')['mort_acc'].mean()
data['mort_acc'] = data.apply(lambda x: total_acc_avg[x['total_acc']] if pd.isnull(x['mort_acc']) else x['mort_acc'], axis=1)


In [ ]:
data['pub_rec_bankruptcies'].fillna(0, inplace=True)


In [ ]:
data['mort_acc_missing'] = data['mort_acc'].isnull().astype(int)


In [ ]:
# Example of one-hot encoding
data = pd.get_dummies(data, columns=['term', 'grade', 'sub_grade', 'home_ownership'], drop_first=True)

# Example of scaling numerical features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_features = ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti', 'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'mort_acc', 'pub_rec_bankruptcies']
data[numeric_features] = scaler.fit_transform(data[numeric_features])


In [ ]:
data['issue_d'] = pd.to_datetime(data['issue_d'])
data['issue_year'] = data['issue_d'].dt.year
data['issue_month'] = data['issue_d'].dt.month


In [ ]:
data = pd.get_dummies(data, columns=['purpose'], drop_first=True)


In [ ]:
data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])
data['credit_age'] = (data['issue_d'] - data['earliest_cr_line']).dt.days // 30  # in months


In [ ]:
data['initial_list_status'] = data['initial_list_status'].map({'w': 0, 'f': 1})


In [ ]:
data['zip_code'] = data['address'].apply(lambda x: x[-5:])  # Assuming ZIP is the last 5 characters
data = pd.get_dummies(data, columns=['zip_code'], drop_first=True)


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Identify numeric columns
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns

# Exclude non-numeric columns
non_numeric_columns = data.columns.difference(numeric_columns)

# Option 1: Scale only the numeric columns
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[numeric_columns])

# Convert back to a DataFrame and retain non-numeric columns
data_scaled_df = pd.DataFrame(data_scaled, columns=numeric_columns)
final_data = pd.concat([data_scaled_df, data[non_numeric_columns].reset_index(drop=True)], axis=1)

# Option 2: If you want to retain non-numeric columns unchanged
# You might prefer to scale only numeric columns and then concatenate with the rest


In [ ]:
# Example of encoding (if grouping or simplification is not performed):
data['emp_title'] = data['emp_title'].astype('category').cat.codes


In [ ]:
mode_imputer = SimpleImputer(strategy='most_frequent')
data['emp_length'] = mode_imputer.fit_transform(data[['emp_length']]).astype(str)
data['title'] = mode_imputer.fit_transform(data[['title']]).astype(str)


In [ ]:
title_counts = data['title'].value_counts()
data['title_encoded'] = data['title'].map(title_counts)


In [ ]:
# Map loan_status to numeric values
data['loan_status'] = data['loan_status'].map({'Fully Paid': 0, 'Charged Off': 1})

In [ ]:
# Compute the mean of the target variable for each title
target_mean = data.groupby('title')['loan_status'].mean()


# Map these means to the original data
data['title'] = data['title'].map(target_mean)


In [ ]:
# Assuming `loan_status` has already been converted to numeric (0 and 1)
emp_title_mean = data.groupby('emp_title')['loan_status'].mean()
data['emp_title'] = data['emp_title'].map(emp_title_mean)


In [ ]:
# Frequency encoding
emp_title_freq = data['emp_title'].value_counts(normalize=True)
data['emp_title'] = data['emp_title'].map(emp_title_freq)


In [ ]:
data = data.drop(['address'], axis=1)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize label encoder
label_encoder = LabelEncoder()

# Apply label encoding to the application_type column
data['application_type'] = label_encoder.fit_transform(data['application_type'])




In [ ]:
# Extracting features from datetime columns
data['issue_year'] = data['issue_d'].dt.year
data['issue_month'] = data['issue_d'].dt.month

# Drop original datetime column if not needed
data = data.drop(columns=['issue_d'])


In [ ]:
# Extracting features from datetime columns
data['earliest_cr_line_year'] = data['earliest_cr_line'].dt.year
data['earliest_cr_line_month'] = data['earliest_cr_line'].dt.month
data = data.drop(columns=['earliest_cr_line'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize label encoder
label_encoder = LabelEncoder()

# Apply label encoding
data['application_type'] = label_encoder.fit_transform(data['application_type'])




In [ ]:
missing_values = data.isnull().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)


In [ ]:
emp_length_mapping = {
    'Unknown': 0,
    '< 1 year': 1,
    '1 year': 2,
    '2 years': 3,
    '3 years': 4,
    '4 years': 5,
    '5 years': 6,
    '6 years': 7,
    '7 years': 8,
    '8 years': 9,
    '9 years': 10,
    '10+ years': 11
}

data['emp_length'] = data['emp_length'].map(emp_length_mapping)


In [ ]:
data['verification_status'] = data['verification_status'].map({'Not Verified': 0, 'Source Verified': 1, 'Verified': 2})

In [ ]:
missing_values = data.isnull().sum()
print("Columns with missing values:\n", missing_values[missing_values > 0])


Columns with missing values:
 Series([], dtype: int64)


In [ ]:
data.to_csv('preprocessed_data.csv', index=False)

In [ ]:
data.to_pickle('preprocessed_data.pkl')
data = pd.read_pickle('preprocessed_data.pkl')

In [ ]:
data = pd.read_csv('preprocessed_data.csv')

In [ ]:
# Select only the numerical features from your data
numerical_data = data.select_dtypes(include=['float64', 'int64'])


In [ ]:
#2.e.1 VIF calculation to check Multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Create a DataFrame to store VIF values
vif_data = pd.DataFrame()
vif_data['Feature'] = numerical_data.columns

# Calculate VIF for each feature
vif_data['VIF'] = [variance_inflation_factor(numerical_data.values, i) for i in range(numerical_data.shape[1])]

# Display the VIF values
print(vif_data)


                   Feature           VIF
0                loan_amnt  1.200614e+01
1                 int_rate  1.299249e+00
2              installment  1.149836e+01
3                emp_title  2.231637e+00
4               emp_length  4.182939e+00
5               annual_inc  1.244228e+00
6      verification_status  3.059915e+00
7              loan_status  1.657010e+00
8                    title  3.768521e+00
9                      dti  1.055770e+00
10                open_acc  2.086534e+00
11                 pub_rec  2.202657e+00
12               revol_bal  1.339278e+00
13              revol_util  1.248655e+00
14               total_acc  2.388642e+00
15     initial_list_status  2.786293e+00
16        application_type  5.602120e+02
17                mort_acc  1.380251e+00
18    pub_rec_bankruptcies  2.002558e+00
19        mort_acc_missing           NaN
20              issue_year  7.078661e+09
21             issue_month  6.679240e+02
22              credit_age  5.216722e+05
23           tit

Interpretation of VIF Results

---
VIF for loan_amnt (**12.00**): Indicates significant multicollinearity. This variable might be highly correlated with other features like installment.

VIF for installment (**11.50**): Also shows significant multicollinearity, likely due to its close relationship with loan_amnt.

VIF for application_type (**560.21**): Extremely high VIF, suggesting severe multicollinearity. This feature may be highly redundant.

VIF for issue_year and earliest_cr_line_year: Extremely high VIF values (in billions), indicating severe multicollinearity. These features are likely highly correlated with each other or other features related to time.

VIF for credit_age (**521,672.20**): This is another example of extreme multicollinearity, which suggests that it’s highly correlated with other time-related variables.

Features with VIF close to 1: These features, like **dti, annual_inc, and int_rate, show low multicollinearity and are generally safe to keep in the model.**

We plan to go ahead with Recursive feature elimination which eliminates  remove less important features and select the ones that contribute most to our model’s performance.


In [ ]:
# 2.e.2 RFE Implementation
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
X = data.drop(columns=['loan_status'])  # Replace 'target_variable' with the actual target column
y = data['loan_status']  # Replace 'target_variable' with the actual target column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Initialize the model
model = LogisticRegression()


In [ ]:
# Initialize RFE with the model and specify the number of features to select
rfe = RFE(model, n_features_to_select=10)


In [ ]:
# Fit RFE
rfe = rfe.fit(X_train, y_train)


In [ ]:
# Print the selected features
selected_features = X_train.columns[rfe.support_]
print("Selected Features by RFE:", selected_features)

Selected Features by RFE: Index(['emp_title', 'title', 'zip_code_11650', 'zip_code_22690',
       'zip_code_29597', 'zip_code_30723', 'zip_code_48052', 'zip_code_70466',
       'zip_code_86630', 'zip_code_93700'],
      dtype='object')


The features selected by RFE are those that the model has determined to have the most predictive power. In our case, features like **emp_title**, **title**, and specific **zip_code** variables were retained because they were deemed most relevant to predicting the target variable.

Model Simplification: By reducing the number of features, RFE simplifies the model, making it easier to interpret and faster to compute, while maintaining (or even improving) model performance.

In [ ]:
label_encoders = {}
ordinal_cols = []  # Specify ordinal columns if any
for col in ordinal_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [ ]:
nominal_cols = ['term', 'grade', 'sub_grade', 'home_ownership',
                'verification_status', 'purpose', 'initial_list_status',
                'application_type', 'emp_title']  # Modify as needed
for col in nominal_cols:
    if col in data.columns and data[col].dtype == 'object':
        data[col] = data[col].str.strip()


In [ ]:
# Calculate the correlation between Loan Amount and Installment
correlation = data['loan_amnt'].corr(data['installment'])
correlation


0.9539289082616188

The value 0.95 suggests that higher loan amounts tend to have higher installments.